## Model Evaluation - Random Forest Models on Diff Data
USA World Series Results,

Run on Contestable Kickoff "bins" only

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#Import Data - USA's differential data
df = pd.read_csv('../data/output/new_features_diffdata.csv')
df.head()

#Import validation data
valdf = pd.read_csv('../data/output/new_features_diffdata_validate.csv')
valdf.head()

,Opp,Tournament,Poss_Time_Diff,Score_Diff,Conv_Diff,Tries_Diff,Passes_Diff,Contestable_KO_Win_pct_Diff,PenFK_Against_Diff,RuckMaul_Diff,...,-99 : -75,-74 : -25,-24 : -1,0 : 25,26 : 50,51 : 75,76 : 100,101 : 125,126 : 150,Result
0,Argentina,2018_Paris_7s,47.004608,57.575758,60.000000,-0.250000,53.846154,80.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,1
1,Canada,2018_London_7s,0.467290,17.391304,25.000000,-0.466667,6.329114,20.000000,100.000000,0.000000,...,0.0,0.0,0.0,12.5,0.0,0.0,0.0,0.0,0.0,1
2,England,2018_London_7s,18.994413,37.777778,42.857143,-0.400000,31.147541,26.666667,-33.333333,0.000000,...,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,1
3,England,2018_Paris_7s,18.461538,0.000000,0.000000,0.000000,35.064935,-50.000000,0.000000,-33.333333,...,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,Fiji,2018_Paris_7s,-6.406685,-60.000000,-60.000000,0.000000,22.222222,-25.000000,-100.000000,-100.000000,...,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## Randomize Data

In [3]:
#Shuffle dataframes before running model to prevent overfitting
from sklearn.utils import shuffle
df = shuffle(df)
valdf = shuffle(valdf)

In [5]:
#Diagnostic
#df.info()
list(df.columns)
#df.head()

['Opp',
 'Tournament',
 'Poss_Time_Diff',
 'Score_Diff',
 'Conv_Diff',
 'Tries_Diff',
 'Passes_Diff',
 'Contestable_KO_Win_pct_Diff',
 'PenFK_Against_Diff',
 'RuckMaul_Diff',
 'Ruck_Win_pct_Diff',
 'Cards_diff',
 'Lineout_Win_Pct_Diff',
 'Scrum_Win_Pct_Diff',
 '-175 : -150',
 '-149 : -125',
 '-124 : -100',
 '-99 : -75',
 '-74 : -25',
 '-24 : -1',
 '0 : 25',
 '26 : 50',
 '51 : 75',
 '76 : 100',
 '101 : 125',
 '126 : 150',
 'Result']

In [4]:
from sklearn.model_selection import train_test_split

In [6]:
#Create a list of features to drop that are unneccessary or will bias the prediction
droplist = ['Opp','Tournament','Poss_Time_Diff','Score_Diff','Conv_Diff','Tries_Diff','Passes_Diff','Contestable_KO_Win_pct_Diff','PenFK_Against_Diff','RuckMaul_Diff','Ruck_Win_pct_Diff','Cards_diff','Lineout_Win_Pct_Diff','Scrum_Win_Pct_Diff']

rf_data = df.drop((droplist), axis=1)

#Drop rows with Result == "T" (Ties). This label messes up classification models
rf_data.drop(rf_data[rf_data.Result == 2].index, inplace=True)

#Pull out the variable we're trying to predict: 'Result'
X = rf_data.drop('Result',axis=1)
y = rf_data['Result']

In [7]:
rf_data.head()
#Check to insure 'Result' only contains 2 values (W, L)
#rf_data['Result'].describe()
#rf_data.describe()

,-175 : -150,-149 : -125,-124 : -100,-99 : -75,-74 : -25,-24 : -1,0 : 25,26 : 50,51 : 75,76 : 100,101 : 125,126 : 150,Result
94,0.0,0.0,0.0,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
0,0.0,0.0,0.0,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0
92,0.0,0.0,0.0,0.0,0.0,0.0,12.5,0.0,0.0,0.0,0.0,0.0,1
153,0.0,0.0,0.0,0.0,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
#list(rf_data.columns) 

In [8]:
#Pull out the variable we're trying to predict: 'Result'
X = rf_data.drop('Result',axis=1)
y = rf_data['Result']
#X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30)

In [10]:
#Split into train/test/validate sets
#OR, keep as is and use new data for validate
#156 rows in original dataframe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

### Validation set
Imported last two series stops (London & Paris, 2018 - 12 matches total) to use as a validation set.

Need to also incorporate into larger data set later.

In [11]:
#Drop features that are unneccessary/str or will bias the prediction
val_data = valdf.drop((droplist), axis=1)

#Drop rows with Result == "T" (Ties). This label messes up classification models
val_data.drop(val_data[val_data.Result == 2].index, inplace=True)

#Pull out the variable we're trying to predict: 'Result'
val_X = val_data.drop('Result',axis=1)
val_y = val_data['Result']

## Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier

#Fit RF Classifier model
rf = RandomForestClassifier(random_state=101)

from pprint import pprint
# Look at parameters used by our current forest
print('Default Parameters currently in use:\n')
pprint(rf.get_params())

Default Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 101,
 'verbose': 0,
 'warm_start': False}


## Evaluate Base Model

In [14]:
from sklearn.metrics import accuracy_score

In [18]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 101)
base_model.fit(X_train, y_train)
predictions = base_model.predict(X_test)
#base_accuracy = evaluate(base_model, y_train, y_test)

#Use sklearn.metrics 'accuracy_score' to determine accuracy of trained and test model
base_train_acc = accuracy_score(y_train, base_model.predict(X_train))
base_test_acc = accuracy_score(y_test, predictions)
print("Base Model")
print('Train:')
print(base_train_acc)
print('Test:')
print(base_test_acc)

Base Model
Train:
0.647619047619
Test:
0.565217391304


### Hyperparameters
* n_estimators = number of trees in the foreset
* max_features = max number of features considered for splitting a node
* max_depth = max number of levels in each decision tree
* min_samples_split = min number of data points placed in a node before the node is split
* min_samples_leaf = min number of data points allowed in a leaf node
* bootstrap = method for sampling data points (with or without replacement)

### Grid Search with Cross Validation
Random search allowed us to narrow down the range for each hyperparameter. Now that we know where to concentrate our search, we can explicitly specify every combination of settings to try. We do this with GridSearchCV, a method that, instead of sampling randomly from a distribution, evaluates all combinations we define.

In [19]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [60, 70, 80, 90, 100, 110],
    'max_features': [4, 5, 6],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'min_samples_split': [2, 5, 8, 10, 12],
    'n_estimators': [100, 200, 300, 400]
}

# Create a base model
#rf = RandomForestClassifier(random_state = 101)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [20]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

best_grid = grid_search.best_estimator_
#grid_accuracy = evaluate(best_grid, X_test, y_test)

print("Base Model")
print(base_train_acc)
print(base_test_acc)

#get predictions with best parameters
grid_predict = best_grid.predict(X_test)

grid_train_acc = accuracy_score(y_train, best_grid.predict(X_train))
grid_test_acc = accuracy_score(y_test, grid_predict)
print("\n")
print("Grid Search Model")
print(grid_train_acc)
print(grid_test_acc)

#print('Improvement of {:0.2f}%.'.format( 100 * (grid_test_acc - base_test_acc) / base_test_acc))

Fitting 5 folds for each of 1800 candidates, totalling 9000 fits
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=2, n_estimators=100, total=   0.3s
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=2, n_estimators=100, total=   0.4s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.1s


[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=5, n_estimators=300, total=   1.0s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=5, n_estimators=400 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=5, n_estimators=300, total=   0.9s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=5, n_estimators=400 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=5, n_estimators=400, total=   1.3s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=5, n_estimators=400 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=5, n_estimators=400, total=   1.2s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samp

[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=10, n_estimators=200, total=   0.9s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=10, n_estimators=300 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=10, n_estimators=200, total=   0.9s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=10, n_estimators=300 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=10, n_estimators=300, total=   1.2s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=10, n_estimators=300 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=10, n_estimators=300, total=   1.1s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=10, n_estimators=400 
[CV]  bootstrap=True, max_depth=60, max_features=4, 

[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=2, min_samples_split=2, n_estimators=100, total=   0.4s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=2, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=1, min_samples_split=12, n_estimators=400, total=   1.6s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=2, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=2, min_samples_split=2, n_estimators=200, total=   0.7s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=2, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=2, min_samples_split=2, n_estimators=200, total=   0.7s
[CV] bootstrap=True, max_depth=60, max_features=4, min_samples_leaf=2, min_samples_split=2, n_estimators=300 
[CV]  bootstrap=True, max_depth=60, max_features=4, min_sam

...

[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=2, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=4, min_samples_split=12, n_estimators=400, total=   1.4s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=2, n_estimators=100, total=   0.5s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=4, min_samples_split=12, n_estimators=400, total=   1.5s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=2, n_estimators=200 
[CV]  bootstrap=True, max_depth=110, max_features=

[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=5, n_estimators=400, total=   1.3s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=5, n_estimators=400, total=   1.3s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=8, n_estimators=100, total=   0.3s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=8, n_estimators=100, total=   0.4s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=110, max_features=6,

[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=10, n_estimators=300 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   1.3s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=10, n_estimators=400 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   1.4s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=10, n_estimators=400 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   1.2s
[CV] bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=10, n_estimators=400 
[CV]  bootstrap=True, max_depth=110, max_features=6, min_samples_leaf=5, min_samples_split=10, n_estimators=300, total=   1.2s
[CV] bootstrap=True, max_depth=110, max_feat

[Parallel(n_jobs=-1)]: Done 9000 out of 9000 | elapsed: 41.2min finished


Base Model
0.647619047619
0.565217391304


Grid Search Model
0.647619047619
0.565217391304


In [21]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_test_acc - base_test_acc) / base_test_acc))

Improvement of 0.00%.


### Grid Search Accuracy Results
**Base Model**
```
1.0
0.45652173913
```
**Grid Search Model**
```
1.0
0.50
```

***Improvement of 9.52%. to 50%***

In [22]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 60,
 'max_features': 4,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [23]:
# examine the best model#
print('Best Estimator:')
print(grid_search.best_estimator_)
print()
print('Best Score:')
print(grid_search.best_score_)
print()
print('Best Parameters:')
print(grid_search.best_params_)

Best Estimator:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=60, max_features=4, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=101, verbose=0, warm_start=False)

Best Score:
0.628571428571

Best Parameters:
{'bootstrap': True, 'max_depth': 60, 'max_features': 4, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


**Best Estimator**
```
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=60, max_features=4, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=101, verbose=0, warm_start=False)
```

**Best Score**
```
0.628571428571
```
**Best Parameters**
```
{'bootstrap': True, 'max_depth': 60, 'max_features': 4, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
```

### Use new parameters from gridsearch to create and fit model

In [25]:
#Fit classifier with new model parameters from gridsearch
rfc = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=60, max_features=4, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=101, verbose=0, warm_start=False)

#Fit model
rfc.fit(X_train, y_train)

#Predict Classifier
rfc_pred = rfc.predict(X_test)

## Random Forest Model Eval

In [26]:
#Accuracy
rfc_acc = accuracy_score(y_test, rfc_pred)
print(rfc_acc)


0.565217391304


In [27]:
#Find Feature Importances
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

print("Feature Importance")
print(feature_importances)

Feature Importance
             importance
-74 : -25      0.296931
26 : 50        0.252913
0 : 25         0.139966
-24 : -1       0.129089
51 : 75        0.041101
-175 : -150    0.000000
-149 : -125    0.000000
-124 : -100    0.000000
-99 : -75      0.000000
76 : 100       0.000000
101 : 125      0.000000
126 : 150      0.000000


In [28]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

#Output confusion matrix
print("Confusion Matrix")
print(confusion_matrix(y_test,rfc_pred))

#import libraries to ignore UndefinedMetricWarning
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

#get the model's accuracy score
accuracy_score(y_test, rfc_pred)
print("\n")
print("Classification Report")
print(classification_report(y_test,rfc_pred))

#print accuracy score
print("\n")
print("Accuracy Score")
print(rfc.score(X_test, y_test))

Confusion Matrix
[[ 4 15]
 [ 5 22]]


Classification Report
             precision    recall  f1-score   support

          0       0.44      0.21      0.29        19
          1       0.59      0.81      0.69        27

avg / total       0.53      0.57      0.52        46



Accuracy Score
0.565217391304


## Predict on Validation Set

In [29]:
#Run Prediction Classifier on validation data (val_X, val_y)
rfc_val_pred = rfc.predict(val_X)

In [30]:
#Accuracy
rfc_val_acc = accuracy_score(val_y, rfc_val_pred)
print(rfc_val_acc)

0.5


In [31]:
#Output confusion matrix
print("Confusion Matrix")
print(confusion_matrix(val_y, rfc_val_pred))

#import libraries to ignore UndefinedMetricWarning
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

#get the model's accuracy score
accuracy_score(val_y, rfc_val_pred)
print("\n")
print("Classification Report")
print(classification_report(val_y, rfc_val_pred))

#print accuracy score
print("\n")
print("Accuracy Score")
print(rfc.score(val_X, val_y))

Confusion Matrix
[[1 3]
 [2 4]]


Classification Report
             precision    recall  f1-score   support

          0       0.33      0.25      0.29         4
          1       0.57      0.67      0.62         6

avg / total       0.48      0.50      0.48        10



Accuracy Score
0.5
